In [1]:
from transformers import BertTokenizer

In [2]:
import torch

In [3]:
import numpy as np
import pandas as pd

import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, Dataset)
from torch.utils.data.distributed import DistributedSampler
from torch import nn

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

In [5]:
import csv

In [6]:
train_data_path = 'train.txt'
# dev_data_path = 'LCQMC/dev'
# test_data_path = 'LCQMC/test'

df = pd.read_csv(train_data_path, sep='\t', header=None, quoting=csv.QUOTE_NONE,encoding='utf-8')

In [7]:
label_list = ['Support','Neutral','Against']
prompt_pattern_list = ['所以','并且','但是']

In [12]:
def truncate_seq_pair(tokens_a, tokens_b, max_num_tokens):
    """Truncates a pair of sequences to a maximum sequence length. Lifted from Google's BERT repo."""
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_num_tokens:
            break

        trunc_tokens = tokens_a if len(tokens_a) > len(tokens_b) else tokens_b
        assert len(trunc_tokens) >= 1

        # We want to sometimes truncate from the front and sometimes from the
        # back to add more randomness and avoid biases.
        if random() < 0.5:
            del trunc_tokens[0]
        else:
            trunc_tokens.pop()

In [17]:
class PromptDataset(Dataset):
    def __init__(self,DATAFRAME,tokenizer,prompt_pattern_list,max_seq_length,label_list):
        self.data = DATAFRAME
        """
        DATAFRAME:为pandas的DataFrame，需要第一列文本
        """
        self.tokenizer = tokenizer
        self.prompt_pattern_list = prompt_pattern_list
        self.label_list = label_list
        self.prompt_pattern_len = len(prompt_pattern_list[0])
        # self.prompt_tokens = tokenizer.tokenize(prompt_pattern)
        # self.prompt_tokens_len = len(self.prompt_tokens)
        self.max_seq_length = max_seq_length
        self.nums_specical_tokens = 2
        self.max_tokens_nums = max_seq_length - self.nums_specical_tokens-self.prompt_pattern_len

    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self,index):
        # text = self.data.iloc[index].strip('\n')
        # text = self.data[0].iloc[index].strip('\n')
        text_a = self.data[0].iloc[index].strip('\n').strip('。')
        text_b = self.data[1].iloc[index].strip('\n').strip('。')
        labels = self.data[2].iloc[index]
        label_ids = self.label_list.index(labels)
        
        tokens_a = self.tokenizer.tokenize(text_a)
        tokens_b = self.tokenizer.tokenize(text_b)

        # tokens = self.tokenizer.tokenize(text)
        truncate_seq_pair(tokens_a, tokens_b, self.max_tokens_nums)
#         tokens = tokens[:self.max_tokens_nums]

        tokens = ["[CLS]"] + tokens_a + ['[MASK]']*self.prompt_pattern_len+ tokens_b + ["[SEP]"]
        # the position of prompt_pattern
        masked_positions = [i for i in range(len(tokens_a)+1,len(tokens_a)+1+self.prompt_pattern_len)]
        # masked_labels = tokens.copy()

        # tokens, masked_lm_positions, masked_lm_labels = create_masked_lm_predictions(tokens, self.masked_lm_prob, self.max_predictions_per_seq, self.vocab_list)

        input_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        # label_ids = self.tokenizer.conver_tokens_to_ids(masked_labels)

        attention_ids = [1] * len(input_ids)
        padding_length = self.max_seq_length - len(input_ids)

        input_ids += [0] * padding_length
        # label_ids += [0] * padding_length

        attention_ids += [0] * padding_length

        segment_ids = [0] * len(input_ids)
        # label_ids = np.full(self.max_seq_length, dtype=np.int, fill_value=-100)
        # label_ids[masked_lm_positions] = self.tokenizer.convert_tokens_to_ids(masked_lm_labels)

        input_ids = torch.tensor(input_ids,dtype=torch.long)
        segment_ids = torch.tensor(segment_ids,dtype=torch.long)
        attention_ids = torch.tensor(attention_ids,dtype=torch.long)
        masked_positions = torch.tensor(masked_positions,dtype=torch.long)
        label_ids = torch.tensor(label_ids,dtype=torch.long)

        return input_ids, segment_ids, attention_ids, masked_positions, label_ids

In [18]:
data = PromptDataset(df,tokenizer,prompt_pattern_list,64,label_list)

In [20]:
input_ids, segment_ids, attention_ids, masked_positions, label_ids=data[0]

In [24]:
attention_ids.size()

torch.Size([64])

In [26]:
input_ids[masked_positions]

tensor([103, 103])

In [4]:
!nvidia-smi

Fri Oct 22 14:16:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.31       Driver Version: 440.31       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN RTX           Off  | 00000000:1B:00.0 Off |                  N/A |
|100%   88C    P2   162W / 280W |  23432MiB / 24220MiB |     99%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN RTX           Off  | 00000000:3E:00.0 Off |                  N/A |
| 61%   81C    P2   269W / 280W |  13084MiB / 24220MiB |     96%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [47]:
!CUDA_VISIBLE_DEVICES=3 python -m torch.distributed.launch --master_port 29501 p_tuning.py

2021-10-22 23:26:40.472496: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.2/lib:
2021-10-22 23:26:40.472538: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
10/22/2021 23:26:45 - INFO - __main__ -   device: cuda:0 n_gpu: 1, distributed training: True, 16-bits training: False
Some weights of the model checkpoint at continues_pretrain_model were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a 

Iteration:  21%|######1                       | 149/722 [00:02<00:09, 62.69it/s]10/22/2021 23:27:15 - INFO - __main__ -   ===================== -epoch 1 -train_step 150 -train_loss 0.2513


Iteration:  27%|########2                     | 198/722 [00:03<00:08, 63.32it/s]10/22/2021 23:27:16 - INFO - __main__ -   ===================== -epoch 1 -train_step 200 -train_loss 0.2487


Iteration:  34%|##########2                   | 247/722 [00:03<00:07, 63.34it/s]10/22/2021 23:27:17 - INFO - __main__ -   ===================== -epoch 1 -train_step 250 -train_loss 0.2540


Iteration:  41%|############2                 | 296/722 [00:04<00:06, 63.25it/s]10/22/2021 23:27:18 - INFO - __main__ -   ===================== -epoch 1 -train_step 300 -train_loss 0.2526


Iteration:  48%|##############3               | 345/722 [00:05<00:05, 63.09it/s]10/22/2021 23:27:19 - INFO - __main__ -   ===================== -epoch 1 -train_step 350 -train_loss 0.2436


Iteration:  55%|################3             | 39

Iteration:  69%|####################6         | 497/722 [00:07<00:03, 62.91it/s]10/22/2021 23:27:34 - INFO - __main__ -   ===================== -epoch 2 -train_step 500 -train_loss 0.2207


Iteration:  76%|######################6       | 546/722 [00:08<00:02, 63.06it/s]10/22/2021 23:27:34 - INFO - __main__ -   ===================== -epoch 2 -train_step 550 -train_loss 0.2202


Iteration:  82%|########################7     | 595/722 [00:09<00:02, 62.97it/s]10/22/2021 23:27:35 - INFO - __main__ -   ===================== -epoch 2 -train_step 600 -train_loss 0.2270


Iteration:  89%|##########################7   | 644/722 [00:10<00:01, 62.84it/s]10/22/2021 23:27:36 - INFO - __main__ -   ===================== -epoch 2 -train_step 650 -train_loss 0.2345


Iteration:  96%|############################7 | 693/722 [00:11<00:00, 62.75it/s]10/22/2021 23:27:37 - INFO - __main__ -   ===================== -epoch 2 -train_step 700 -train_loss 0.2294


Iteration: 100%|##############################| 72

============================ -epoch 3 -train_loss 0.2079 -eval_acc 0.9065 -best_acc 0.9065

Iteration:   6%|#8                             | 44/722 [00:00<00:11, 60.87it/s]10/22/2021 23:27:57 - INFO - __main__ -   ===================== -epoch 4 -train_step 50 -train_loss 0.1578


Iteration:  13%|###9                           | 93/722 [00:01<00:10, 62.27it/s]10/22/2021 23:27:58 - INFO - __main__ -   ===================== -epoch 4 -train_step 100 -train_loss 0.1895


Iteration:  21%|######1                       | 149/722 [00:02<00:09, 62.92it/s]10/22/2021 23:27:59 - INFO - __main__ -   ===================== -epoch 4 -train_step 150 -train_loss 0.1979


Iteration:  27%|########2                     | 198/722 [00:03<00:08, 62.90it/s]10/22/2021 23:28:00 - INFO - __main__ -   ===================== -epoch 4 -train_step 200 -train_loss 0.1842


Iteration:  34%|##########2                   | 247/722 [00:04<00:07, 62.95it/s]10/22/2021 23:28:01 - INFO - __main__ -   ===================== -epoc

Iteration:  48%|##############2               | 343/722 [00:05<00:05, 63.18it/s]10/22/2021 23:28:15 - INFO - __main__ -   ===================== -epoch 5 -train_step 350 -train_loss 0.1751


Iteration:  55%|################5             | 399/722 [00:06<00:05, 62.94it/s]10/22/2021 23:28:16 - INFO - __main__ -   ===================== -epoch 5 -train_step 400 -train_loss 0.1714


Iteration:  62%|##################6           | 448/722 [00:07<00:04, 63.08it/s]10/22/2021 23:28:16 - INFO - __main__ -   ===================== -epoch 5 -train_step 450 -train_loss 0.1762


Iteration:  69%|####################6         | 497/722 [00:07<00:03, 62.70it/s]10/22/2021 23:28:17 - INFO - __main__ -   ===================== -epoch 5 -train_step 500 -train_loss 0.1755


Iteration:  76%|######################6       | 546/722 [00:08<00:02, 63.12it/s]10/22/2021 23:28:18 - INFO - __main__ -   ===================== -epoch 5 -train_step 550 -train_loss 0.1775


Iteration:  82%|########################7     | 59


Iteration:  20%|######1                       | 147/722 [00:02<00:09, 62.81it/s]10/22/2021 23:28:24 - INFO - __main__ -   ===================== -epoch 6 -train_step 150 -train_loss 0.1390


Iteration:  27%|########1                     | 196/722 [00:03<00:08, 63.09it/s]10/22/2021 23:28:25 - INFO - __main__ -   ===================== -epoch 6 -train_step 200 -train_loss 0.1571


Iteration:  34%|##########1                   | 245/722 [00:03<00:07, 63.17it/s]10/22/2021 23:28:26 - INFO - __main__ -   ===================== -epoch 6 -train_step 250 -train_loss 0.1630


Iteration:  41%|############2                 | 294/722 [00:04<00:06, 63.01it/s]10/22/2021 23:28:27 - INFO - __main__ -   ===================== -epoch 6 -train_step 300 -train_loss 0.1667


Iteration:  48%|##############2               | 343/722 [00:05<00:05, 63.17it/s]10/22/2021 23:28:28 - INFO - __main__ -   ===================== -epoch 6 -train_step 350 -train_loss 0.1764


Iteration:  55%|################5             | 3

Iteration:  62%|##################6           | 448/722 [00:07<00:04, 63.13it/s]10/22/2021 23:28:45 - INFO - __main__ -   ===================== -epoch 7 -train_step 450 -train_loss 0.1725


Iteration:  69%|####################6         | 497/722 [00:07<00:03, 62.99it/s]10/22/2021 23:28:45 - INFO - __main__ -   ===================== -epoch 7 -train_step 500 -train_loss 0.1736


Iteration:  76%|######################6       | 546/722 [00:08<00:02, 63.09it/s]10/22/2021 23:28:46 - INFO - __main__ -   ===================== -epoch 7 -train_step 550 -train_loss 0.1729


Iteration:  82%|########################7     | 595/722 [00:09<00:02, 63.14it/s]10/22/2021 23:28:47 - INFO - __main__ -   ===================== -epoch 7 -train_step 600 -train_loss 0.1750


Iteration:  89%|##########################7   | 644/722 [00:10<00:01, 63.39it/s]10/22/2021 23:28:48 - INFO - __main__ -   ===================== -epoch 7 -train_step 650 -train_loss 0.1733


Iteration:  96%|############################7 | 69

Evaluating: 100%|###############################| 81/81 [00:01<00:00, 69.04it/s]
============================ -epoch 8 -train_loss 0.1570 -eval_acc 0.9112 -best_acc 0.9159

Iteration:   7%|##1                            | 49/722 [00:00<00:10, 63.17it/s]10/22/2021 23:29:04 - INFO - __main__ -   ===================== -epoch 9 -train_step 50 -train_loss 0.1707


Iteration:  14%|####2                          | 98/722 [00:01<00:09, 63.11it/s]10/22/2021 23:29:04 - INFO - __main__ -   ===================== -epoch 9 -train_step 100 -train_loss 0.1436


Iteration:  20%|######1                       | 147/722 [00:02<00:09, 62.96it/s]10/22/2021 23:29:05 - INFO - __main__ -   ===================== -epoch 9 -train_step 150 -train_loss 0.1437


Iteration:  27%|########1                     | 196/722 [00:03<00:08, 62.97it/s]10/22/2021 23:29:06 - INFO - __main__ -   ===================== -epoch 9 -train_step 200 -train_loss 0.1459


Iteration:  34%|##########1                   | 245/722 [00:03<00:07

Iteration:  48%|##############2               | 343/722 [00:05<00:06, 63.07it/s]10/22/2021 23:29:21 - INFO - __main__ -   ===================== -epoch 10 -train_step 350 -train_loss 0.1467


Iteration:  55%|################5             | 399/722 [00:06<00:05, 63.05it/s]10/22/2021 23:29:22 - INFO - __main__ -   ===================== -epoch 10 -train_step 400 -train_loss 0.1444


Iteration:  62%|##################6           | 448/722 [00:07<00:04, 62.82it/s]10/22/2021 23:29:23 - INFO - __main__ -   ===================== -epoch 10 -train_step 450 -train_loss 0.1484


Iteration:  69%|####################6         | 497/722 [00:07<00:03, 63.12it/s]10/22/2021 23:29:23 - INFO - __main__ -   ===================== -epoch 10 -train_step 500 -train_loss 0.1434


Iteration:  76%|######################6       | 546/722 [00:08<00:02, 62.89it/s]10/22/2021 23:29:24 - INFO - __main__ -   ===================== -epoch 10 -train_step 550 -train_loss 0.1432


Iteration:  82%|########################7    

In [39]:
!CUDA_VISIBLE_DEVICES=3 python -m torch.distributed.launch --master_port 29503 pretrain.py

2021-10-22 18:18:37.345692: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.2/lib:
2021-10-22 18:18:37.345736: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
10/22/2021 18:18:41 - INFO - __main__ -   device: cuda:0 n_gpu: 1, distributed training: True, 16-bits training: False
Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing 

Iteration:  16%|####6                        | 199/1250 [00:31<02:43,  6.42it/s]10/22/2021 18:19:28 - INFO - __main__ -   ===================== -epoch 0 -train_step 200 -train_loss 0.5866


Iteration:  24%|######9                      | 299/1250 [00:47<02:28,  6.41it/s]10/22/2021 18:19:44 - INFO - __main__ -   ===================== -epoch 0 -train_step 300 -train_loss 0.5425


Iteration:  29%|########4                    | 364/1250 [00:58<02:24,  6.11it/s]


Iteration:  32%|#########2                   | 399/1250 [01:03<02:13,  6.37it/s]10/22/2021 18:20:00 - INFO - __main__ -   ===================== -epoch 0 -train_step 400 -train_loss 0.5071


Iteration:  40%|###########5                 | 499/1250 [01:19<01:57,  6.37it/s]10/22/2021 18:20:16 - INFO - __main__ -   ===================== -epoch 0 -train_step 500 -train_loss 0.4811


Iteration:  44%|############8                | 556/1250 [01:28<01:53,  6.14it/s]


Iteration:  48%|#############8               | 599/1250 [01:35<01:42,  6.36it/s]10/22/2021 18:20:32 - INFO - __main__ -   ===================== -epoch 0 -train_step 600 -train_loss 0.4619


Iteration:  56%|################2            | 699/1250 [01:51<01:27,  6.30it/s]10/22/2021 18:20:48 - INFO - __main__ -   ===================== -epoch 0 -train_step 700 -train_loss 0.4440


Iteration:  60%|#################3           | 748/1250 [01:59<01:21,  6.14it/s]


Iteration:  64%|##################5          | 799/1250 [02:07<01:11,  6.30it/s]10/22/2021 18:21:04 - INFO - __main__ -   ===================== -epoch 0 -train_step 800 -train_loss 0.4294


Iteration:  72%|####################8        | 899/1250 [02:23<00:55,  6.36it/s]10/22/2021 18:21:20 - INFO - __main__ -   ===================== -epoch 0 -train_step 900 -train_loss 0.4161


Iteration:  75%|#####################8       | 940/1250 [02:30<00:50,  6.13it/s]


...

Iteration:  96%|##########################8 | 1199/1250 [03:24<00:08,  6.00it/s]10/22/2021 19:25:56 - INFO - __main__ -   ===================== -epoch 17 -train_step 1200 -train_loss 0.0539


Iteration: 100%|############################| 1250/1250 [03:32<00:00,  5.87it/s]

Evaluating:  82%|#######################8     | 457/556 [00:07<00:01, 58.82it/s]


Evaluating: 100%|#############################| 556/556 [00:09<00:00, 58.20it/s]
The eval loss is decreasing!,so we save model!
============================ -epoch 17 -train_loss 0.0540 -eval_loss 0.1126

Iteration:   8%|##3                           | 99/1250 [00:16<03:12,  5.99it/s]10/22/2021 19:26:33 - INFO - __main__ -   ===================== -epoch 18 -train_step 100 -train_loss 0.0538


Iteration:  14%|####                         | 174/1250 [00:29<03:07,  5.75it/s]


Iteration:  16%|####6                        | 199/1250 [00:33<02:55,  5.99it/s]10/22/2021 19:26:50 - INFO - __main__ -   ===================== -epoch 18 -train_step 200 -train_loss 0.0528


Iteration:  24%|######9                      | 299/1250 [00:50<02:38,  5.99it/s]10/22/2021 19:27:07 - INFO - __main__ -   ===================== -epoch 18 -train_step 300 -train_loss 0.0552


Iteration:  29%|########4                    | 366/1250 [01:02<02:33,  5.74it/s]


Iteration:  32%|#########2                   | 399/1250 [01:07<02:21,  6.00it/s]10/22/2021 19:27:24 - INFO - __main__ -   ===================== -epoch 18 -train_step 400 -train_loss 0.0545


Iteration:  40%|###########5                 | 499/1250 [01:24<02:05,  5.98it/s]10/22/2021 19:27:41 - INFO - __main__ -   ===================== -epoch 18 -train_step 500 -train_loss 0.0546


Iteration:  45%|############9                | 558/1250 [01:34<02:00,  5.74it/s]


Iteration:  46%|#############3               | 577/1250 [01:38<01:52,  5.98it/s]^C
Traceback (most recent call last):
  File "/root/anaconda3/envs/pytorch/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/root/anaconda3/envs/pytorch/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/root/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/distributed/launch.py", line 260, in <module>
    main()
  File "/root/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/distributed/launch.py", line 253, in main
    process.wait()
  File "/root/anaconda3/envs/pytorch/lib/python3.6/subprocess.py", line 1477, in wait
    (pid, sts) = self._try_wait(0)
  File "/root/anaconda3/envs/pytorch/lib/python3.6/subprocess.py", line 1424, in _try_wait
    (pid, sts) = os.waitpid(self.pid, wait_flags)
KeyboardInterrupt
Epoch:  90%|#############################7   | 18/20 [1:08:58<07:39, 229.91s/it]
Traceback (most recent call 

In [1]:
!CUDA_VISIBLE_DEVICES=3 python train.py

2021-10-23 12:55:50.584840: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.2/lib:
2021-10-23 12:55:50.584886: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Some weights of the model checkpoint at continues_pretrain_model were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a

Epochs 0/2. Running Loss:    0.1442:  22%|#1   | 80/361 [00:07<00:27, 10.39it/s]


Epochs 0/2. Running Loss:    0.2237:  36%|#4  | 129/361 [00:12<00:22, 10.34it/s]


Epochs 0/2. Running Loss:    0.1969:  49%|#9  | 177/361 [00:17<00:17, 10.37it/s]


Epochs 0/2. Running Loss:    0.2657:  63%|##5 | 226/361 [00:22<00:12, 10.45it/s]


...

Epochs 1/2. Running Loss:    0.3347:  54%|##1 | 196/361 [00:23<00:19,  8.40it/s]


Epochs 1/2. Running Loss:    0.1036:  68%|##7 | 244/361 [00:29<00:14,  8.35it/s]


Epochs 1/2. Running Loss:    0.3488:  81%|###2| 293/361 [00:34<00:08,  8.42it/s]


Epochs 1/2. Running Loss:    0.0056:  94%|###7| 341/361 [00:40<00:02,  8.46it/s]


Epochs 1/2. Running Loss:    0.0051: 100%|####| 361/361 [00:43<00:00,  8.39it/s]
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_3_3
INFO:simpletransformers.classification.classification_model:{'mcc': 0.8904577526081328, 'clf_report': {'0.0': {'precision': 0.9424778761061947, 'recall': 0.8875, 'f1-score': 0.9141630901287554, 'support': 240}, '1.0': {'precision': 0.8783783783783784, 'recall': 0.9285714285714286, 'f1-score': 0.9027777777777779, 'support': 210}, '2.0': {'precision': 0.9639175257731959, 'recall': 0.9739583333333334, 'f1-score': 0.9689119170984456, 'support': 192}, 'accuracy': 0.926791277258567, 'macro avg': {'precision': 0.9282579267525897, 'recall': 0.9300099206349207, 'f1-score': 0.9286175950016596, 'support': 642}, 'weighted avg': {'precision': 0.9279226085255449, 'recall': 0.926791277

In [2]:
!CUDA_VISIBLE_DEVICES=3 python test.py

2021-10-23 12:58:41.188759: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.2/lib:
2021-10-23 12:58:41.188807: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_3_3
Running Evaluation: 100%|#####################| 124/124 [00:01<00:00, 68.04it/s]
INFO:simpletransformers.classification.classification_model:{'mcc': 0.662764195529488, 'clf_report': {'0.0': {'precision': 0.7, 'recall': 0.7821229050279329, 'f1-score': 0.7387862796833773, 'support': 358}, '1.0': {'precision': 0.8, 'recall': 0.62

In [ ]:
!zip 